<a href="https://colab.research.google.com/github/rjhalliday/python-llm/blob/main/langchain_simple_pdf_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a simple langchain rag. We load all documents in the pdf folder, however in this specific example I've only placed a single PDF in the folder as otherwise loading the PDFs takes a very long time due to the free Fremini rate limit.

In [1]:
# rh
!pip install -qU langchain-google-genai langchain-community chromadb pymupdf pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00

In [28]:
# Langchain dependencies
from langchain.document_loaders.pdf import PyPDFDirectoryLoader # Importing PDF loader from Langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter # Importing text splitter from Langchain
from langchain.embeddings import OpenAIEmbeddings # Importing OpenAI embeddings from Langchain
from langchain.schema import Document # Importing Document schema from Langchain
from langchain.vectorstores.chroma import Chroma # Importing Chroma vector store from Langchain
from dotenv import load_dotenv # Importing dotenv to get API key from .env file
from langchain.chat_models import ChatOpenAI # Import OpenAI LLM
import os # Importing os module for operating system functionalities
import shutil # Importing shutil module for high-level file operations


from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
from langchain.prompts import ChatPromptTemplate

import textwrap
from pprint import pprint
import json


In [3]:
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Initialize the language model using GoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Initialize embeddings
embeddings = GooglePalmEmbeddings()

In [4]:
# Directory to your pdf files:
DATA_PATH = "/content/drive/MyDrive/pdfs"
def load_documents():
  """
  Load PDF documents from the specified directory using PyPDFDirectoryLoader.
  Returns:
  List of Document objects: Loaded PDF documents represented as Langchain
                                                          Document objects.
  """
  # Initialize PDF loader with specified directory
  document_loader = PyPDFDirectoryLoader(DATA_PATH)
  # Load PDF documents and return them as a list of Document objects
  return document_loader.load()

documents = load_documents() # Call the function
# Inspect the contents of the first document as well as metadata
print(documents[0])

page_content=' 
 Data -Over -Cable Service Interface Specifications 
DOCSIS 2.0 + IPv6 Cable Modem Specification  
CM-SP-DOCSIS2.0 -IPv6 -I07-130404  
ISSUED  
Notice  
This DOCSIS ® specification is the result of a cooperative effort 
undertaken at the direction of Cable Television Laboratories, Inc. for the 
benefit of the cable industry and its customers. This document may 
contain references to other documents not owned or controlled by CableLabs ®. Use and understanding of this document may require 
access to such other documents. Designing, manufacturing, distributing, 
using, selling, or servicing products, or providing services, based on this 
document may require intellectual property licenses from third parties for technology referenced in this document.  
Neither CableLabs nor any member company is responsible to any 
party for any liability of any nature whatsoever resulting from or arising 
out of use or reliance upon this document, or any document referenced 
herein. This

# chuncking

In [5]:
def split_text(documents: list[Document]):
  """
  Split the text content of the given list of Document objects into smaller chunks.
  Args:
    documents (list[Document]): List of Document objects containing text content to split.
  Returns:
    list[Document]: List of Document objects representing the split text chunks.
  """
  # Initialize text splitter with specified parameters
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300, # Size of each chunk in characters
    chunk_overlap=100, # Overlap between consecutive chunks
    length_function=len, # Function to compute the length of the text
    add_start_index=True, # Flag to add start index to each chunk
  )

  # Split documents into smaller chunks using text splitter
  chunks = text_splitter.split_documents(documents)
  print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

  # Print example of page content and metadata for a chunk
  document = chunks[0]
  print(document.page_content)
  print(document.metadata)

  return chunks # Return the list of split text chunks

In [6]:
# Path to the directory to save Chroma database - save it on google drive
#CHROMA_PATH = "/content/drive/MyDrive/chroma"
CHROMA_PATH = "chroma"
COLLECTION_NAME="text-collection"

def save_to_chroma(chunks: list[Document]):
    """
    Save the given list of Document objects to a Chroma database, updating existing data if present.
    Args:
    chunks (list[Document]): List of Document objects representing text chunks to save.
    Returns:
    None
    """
    # Check if the Chroma vector store exists
    if os.path.exists(CHROMA_PATH):
        # Load the existing Chroma database
        client = Chroma.load(persist_directory=CHROMA_PATH, collection_name=COLLECTION_NAME)
    else:
        # Create a new Chroma database if it doesn't exist
        client = Chroma.from_documents(
            [],
            embeddings,
            persist_directory=CHROMA_PATH,
            collection_name=COLLECTION_NAME
        )

    # Add new documents to the existing or new Chroma database
    client.add_documents(chunks)

    # Persist the updated database to disk
    client.persist()
    print(f"Updated Chroma database with {len(chunks)} new chunks at {CHROMA_PATH}.")
    return client

In [7]:
def generate_data_store():
  """
  Function to generate vector database in chroma from documents.
  """
  documents = load_documents() # Load documents from a source
  chunks = split_text(documents) # Split documents into manageable chunks
  client = save_to_chroma(chunks) # Save the processed data to a data store

  return client

# Load environment variables from a .env file
load_dotenv()
# Generate the data store
client = generate_data_store()

Split 26 documents into 280 chunks.
Data -Over -Cable Service Interface Specifications 
DOCSIS 2.0 + IPv6 Cable Modem Specification  
CM-SP-DOCSIS2.0 -IPv6 -I07-130404  
ISSUED  
Notice  
This DOCSIS ® specification is the result of a cooperative effort 
undertaken at the direction of Cable Television Laboratories, Inc. for the
{'source': '/content/drive/MyDrive/pdfs/CM-SP-DOCSIS2.0-IPv6-I07-130404.pdf', 'page': 0, 'start_index': 3}
Saved 280 chunks to chroma.


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [8]:
query_text = "List the CM interfaces"

In [9]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
 - -
Answer the question based on the above context: {question}
"""

In [44]:
def query_rag(query_text):
  """
  Query a Retrieval-Augmented Generation (RAG) system using Chroma database and Germini.
  Args:
    - query_text (str): The text to query the RAG system with.
  Returns:
    - formatted_response (str): Formatted response including the generated text and sources.
    - response_text (str): The generated response text.
  """
  # YOU MUST - Use same embedding function as before

  # Prepare the database
  #db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)

  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings, collection_name=COLLECTION_NAME)

  #docs = db.similarity_search(query_text)
#  collection = db.get()

#  print("collection")
#  print(collection)

  #collection = db.get_or_create_collection(COLLECTION_NAME)

  # Retrieving the context from the DB using similarity search
  results = db.similarity_search_with_relevance_scores(query_text, k=3)

  #print( f"results:")
  #pprint(results)

  #wrapped_results = textwrap.fill(str(results),width=80)
  #print(wrapped_results)

  # Check if there are any matching results or if the relevance score is too low
  #print(f"results:{len(results)}")
  if len(results) == 0:
    print(f"No matching results found.")
    return

  # Extract similarity scores from results
  similarity_scores = [score for _, score in results]

  print(f"similarity_scores{similarity_scores}")

  # Find the lowest similarity score
  highest_similarity = max(similarity_scores)

  if highest_similarity < 0.7:
    print(f"Warning: Only poor matches to query have been found in vector store. Highest similirity is {highest_similarity: .0%}")

  # Combine context from matching documents
  context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])

  # Create prompt template using context and query text
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context_text, question=query_text)

  # Initialize OpenAI chat model
  model = llm

  #print(prompt)

  # Generate response text based on the prompt
  response_text = model.invoke(prompt)

   # Get sources of the matching documents
  sources = [doc.metadata.get("source", None) for doc, _score in results]

  #print(f"sources:{sources}")

  # Format and return response including generated text and sources
  formatted_response = f"Response: {response_text}\nSources: {sources}"
  return formatted_response, response_text

# Let's call our function we have defined
#formatted_response, response_text = query_rag("What does the term CMCI mean?")
# and finally, inspect our final response!
#print(response_text.content)


In [45]:
# Let the user query the documents:
while (True):
  user_query = input("Type in your question: ")
  if (user_query == "exit"):
    break
  formatted_response, response_text = query_rag(user_query)
  #print(formatted_response)
  print(response_text.content)
  print("---------------")


Type in your question: What does CM mean?
similarity_scores[0.6206714934656465, 0.5110424018398245, 0.49300320313640067]
Based on the provided text, CM most likely stands for **Cable Modem**. 

Here's why:

* **"IF3-MIB"** refers to an SNMP Management Information Base (MIB) related to cable modem interfaces.
* **"docsIfCmStatusValue"** and **"docsIfCmtsCmStatusValue"** suggest objects related to the status of a CM and CMTS (Cable Modem Termination System).
* The reference to **"Data-Over-Cable Service Interface Specifications"** further strengthens the connection to cable modem technology. 

Therefore, considering the context, CM most likely refers to Cable Modem. 

---------------
Type in your question: What does CMCI mean?
similarity_scores[0.5544772547746797, 0.5454541917084196, 0.530824780565154]
Based on the provided text, CMCI stands for **Customer-premises equipment Metropolitan Area Network Circuit Interface**. 

The text states: "The term 'CMCI port' describes physical interfa